In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
from torchvision.models import vit_b_16

model = vit_b_16(weights=None)

model

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [22]:
from vitssm.models.video_transformer import LatentNextFramePrediction
import xformers
from torch import nn
import torch


patchify = nn.Conv2d(3, 128, kernel_size=4, stride=4)
pos_enc = SinePositionalEmbedding(128)

x = torch.zeros((32, 30, 32, 32, 3))
b, t, h, w, c = x.shape
x = rearrange(x, "b t h w c -> (b t) c h w")
print(x.shape)
x = patchify(x)
print(x.shape)
x = rearrange(x, "(b t) e h w -> (b t) (h w) e", b=b, t=t)
print(x.shape)
x = pos_enc(x)

torch.Size([960, 3, 32, 32])
torch.Size([960, 128, 8, 8])
torch.Size([960, 64, 128])


In [12]:
from xformers.components.positional_embedding import SinePositionalEmbedding
from einops import rearrange

pos_enc = SinePositionalEmbedding(3)

x = torch.zeros((32, 30, 224, 224, 3))
x = rearrange(x, "b t h w c -> b (t h w) c")

print(x.shape)
print(pos_enc(x).shape)

torch.Size([32, 1505280, 3])
torch.Size([32, 1505280, 3])


In [3]:
from omegaconf import OmegaConf
from pathlib import Path

base_config = OmegaConf.load(Path("run_configs") / "base_config.yml")
config = OmegaConf.load(Path("run_configs") / "config_run_1.yml")
conf = OmegaConf.merge(base_config, config)

In [1]:
from torchvision.datasets import MovingMNIST
from vitssm.data.datasets import NextFrameDataset


mnist = MovingMNIST(root="data", split="train", split_ratio=10, download=False)
data = NextFrameDataset(mnist)


(tensor([[[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         ...,
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
      

In [2]:
print(data[0][0].shape, data[0][1].shape)

torch.Size([9, 1, 64, 64]) torch.Size([9, 1, 64, 64])


In [5]:
import xformers
import torch
torch.cuda.is_available()

True

In [6]:
from vitssm.engine.tasks import ActionRecognitionEngine
import torch

model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
engine = ActionRecognitionEngine(model, conf)

ModuleNotFoundError: No module named 'addict'

In [ ]:
from vitssm.utils.metrics import MetricContainer
from torchmetrics import Accuracy, Precision, Recall
import torch

x = torch.randn(100, 100)
y = torch.randint(0, 2, (100, 100))
metric_col(x, y)

In [ ]:
from vitssm.engine import build_optimizer
from addict import Dict
from torch import nn

config = Dict({
    "optimization": {
        "optimizer": {
            "name": "Adam",
            "args": {
                "lr": 0.001,
                "weight_decay": 0.0
            }
        }
    }    
})

model = nn.Linear(10, 10)
build_optimizer(model, config)

In [ ]:
from vitssm.data import build_dataset
from addict import Dict
import kappaconfig as kc
import os
from pathlib import Path

config = kc.from_file_uri(Path("run_configs") / "base_config.yml")
print(config.keys())
res = kc.DefaultResolver()
config = Dict(res.resolve(config))

dataset = build_dataset(config)

In [ ]:
dataset[1000][2]

In [ ]:
print(os.getcwd())
os.chdir()

In [ ]:
dataset

In [ ]:
train_dataset[0].permute(0, 2, 3, 1).repeat(1, 1, 1, 3).shape

In [ ]:
from torchvision.io import write_video

write_video("test.mp4", train_dataset[0].permute(0, 2, 3, 1).repeat(1, 1, 1, 3), fps=10)